In [ ]:
print("Hello World!")

In [ ]:
#Autoencoder
class AutoEncoder(nn.Module):
    def __init__(self):
        self.name = "AutoEncoder"
        super(AutoEncoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.ReLU(),
            nn.Linear(57, 40),
        )
        self.decoder = nn.Sequential(
            nn.Linear(40, 57),
            nn.ReLU(),
            nn.Sigmoid() 
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x